Data and codes for the copula selection results (table 3). 

In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', None)
import sys, os
sys.path.append('../../src/')
# import toolbox
from toolbox import *

In [4]:
# Choose copula by AIC
result_names = ['BBT_Tiingo',
                'BBT_future_Tiingo_ada', 
                'BBT_future_Tiingo_eth',
                'BBT_future_Tiingo_ltc',
                'BBT_future_Tiingo_xrp',
                'BBT_future_BITX',
                'BBT_future_BITW20', 
                'BBT_future_BITW70', 
                'BBT_future_BITW100', 
                'BBT_future_CRIX']

coin_names = ['btc', 'ada', 'eth', 'ltc', 'xrp',
              'BITX', 'BITW20', 'BITW70', 'BITW100', 'CRIX']

NIG_coin_names =  ['btc', 'ada', 'eth', 'ltc', 'xrp',
              'bitx', 'bitw20', 'bitw70', 'bitw100', 'CRIX']

result_coin_mapper = dict(zip(result_names, coin_names))

In [5]:
k = {'Gaussian':1,
     't_Copula':2, 
     't_Copula_Capped':2, 
     'Clayton':1, 
     'Frank':1, 
     'Gumbel':1,  
     'Plackett':1,  
     'Gauss Mix Indep':2, 
     'rotGumbel':1, 
     'NIG':3}

copula_names = list(k.keys())

def AIC_table(l, drop_t_Copula_Capped=True):
    # l likelihood table
    if drop_t_Copula_Capped:
        l = l.drop(index='t_Copula_Capped')
       
    copula_names = list(l.index)
    AIC = l.copy()
    
    for c in copula_names:
        AIC.loc[c,:] = -2*l.loc[c,:]+2*k[c]
        
    return AIC

In [6]:
AIC_results = dict()
copula_to_use = dict()
copula_count = dict()

for result in result_names:
    l = pd.read_json('../../results/%s/MM/likelihood_NIG.json'%result) # likelihood 
#     l *=300 # 300 data points
    
    _AIC = AIC_table(l, True)
    AIC_results[result] = _AIC
    
    _copula = _AIC.idxmin(axis='rows')
    _copula = pd.DataFrame(_copula).reset_index()
    _copula.columns = ['file', 'copula']
    copula_to_use[result] = _copula
    
    copula_count[result]= pd.pivot_table(_copula, index=['copula'], aggfunc='count')

In [11]:
copula_count_summary = pd.DataFrame(columns=copula_names)

for rm in result_names:
    c = copula_count[rm]
    c.index.name = None
    c.columns = [result_coin_mapper[rm]]
    copula_count_summary = copula_count_summary.append(c.T)
    
copula_count_summary = copula_count_summary.fillna(0)

copula_count_summary = pd.concat([copula_count_summary.loc[:,c] for c in copula_names if c != 't_Copula_Capped'],
          axis=1)

copula_count_summary.loc['sum',:] = copula_count_summary.sum()